测试组合：

	第一类组合：全市场股票按市值排序，划分为N个股票数目为10个的组合</p>

	第二类组合：全市场股票按市值排序，划分为N个股票数目为50个的组合</p>

	第三类组合：全市场股票按市值排序，划分N个股票数目为100个的组合</p>

	第四类组合：申万一级行业分类的28个行业作为股票组合</p>

	第五类组合：买入申万一级行业分类的28个行业中：（1）各行业中市值最大的股票，共计28只股票；(2) 各行业中市值最小的股票，共计28只股票</p>

	第六类组合：买入申万一级行业分类的28个行业中：（1）各行业中市值前二的股票，共计56只股票；(2) 各行业中市值后二的股票，共计56只股票</p>

	第七类组合：买入申万一级行业分类的28个行业中：（1）各行业中市值前五股票，共计140只股票；(2) 各行业中市值后五股票，共计140只股票</p>

	第八类组合：买入申万一级行业分类的28个行业中：（1）各行业中市值前五和市值后五的股票，共计280只股票 </p>



In [ ]:
def get_subnew_delisted_assets(order_book_ids,date,N,type="CS"):
    """
    # 获得某日上市小于N天的次新股
    :param stocks: list 股票列表
    :param date: str eg. "2018-01-01"
    :param N: int 次新股过滤的阈值
    :return: list 列表中的次新股
    """

    instruments_fun = fund.instruments if type == "Fund" else instruments
    all_detail_instruments = instruments_fun(order_book_ids)
    subnew_assets = [s for s in all_detail_instruments if len(get_trading_dates(s.listed_date,date))<=N] if isinstance(N,int) else []
    delisted_assets = [s for s in all_detail_instruments if (not s.de_listed_date =="0000-00-00") and pd.Timestamp(s.de_listed_date)<pd.Timestamp(date)]

    return subnew_assets,delisted_assets

def get_st_stocks(stocks,date):
    """
    获得某日的ST类股票
    :param stocks: list 股票列表
    :param date: 交易日
    :return: list 列表中st的股票
    """
    previous_date = get_previous_trading_date(date)
    st_series = is_st_stock(stocks,start_date=previous_date,end_date=date).iloc[-1]
    return st_series[st_series].index.tolist()

def get_suspended_stocks(stocks,end_date,N):
    """
    获得起始日期内未停牌过的股票列表
    :param stocks: list 股票列表
    :param start_date: 交易日
    :return: list 列表中的停牌股
    """
    start_date = rqdatac.trading_date_offset(end_date,-N)
    volume = get_price(stocks, start_date, end_date, fields="volume")
    suspended_day = (volume == 0).sum(axis=0)
    return suspended_day[suspended_day>0].index.tolist()

####  以下获得剔除ST、次新股、退市、停牌股票列表 （次新股阈值=126）

#### 获得 2014-2018年6月31日的所有月份的第一个交易日

In [ ]:
dates = get_trading_dates("2014-01-01","2018-06-31")
df = pd.DataFrame(dates).astype(str)
df["group"] = df[0].str[:7]
dates = df.groupby("group").first()[0].values.tolist()

In [ ]:
window=126
stocks_list = {}
for d in dates[:]:
    stocks = all_instruments("CS",date=d).order_book_id.tolist()
    st = get_st_stocks(stocks,d)
    subnew,delisted = get_subnew_delisted_assets(stocks,d,window)
    suspend = get_suspended_stocks(stocks,d,window)
    stocks = list(set(stocks)-set(st)-set(subnew)-set(suspend)-set(delisted))
    stocks_list[d] = stocks
pd.Series(stocks_list).to_pickle("all_stocks_list.pkl")

#### 获得测试组合

In [5]:
import pandas as pd
import numpy as np

In [6]:
data = pd.read_pickle("all_stocks_list.pkl")

#### plan1-3

In [ ]:
def get_stocks_list(stocksList,size,d):
    marketCap = get_factor(stocksList,"a_share_market_val",date=d)  
    marketCap.sort_values(ascending=False,inplace=True)
    cut_stocks={}
    for i in np.arange(int(len(marketCap)/size)):
        cut_stocks[i] = (marketCap.truncate(i*size,(i+1)*size)).index.tolist()
    return pd.Series(cut_stocks)

In [ ]:
size = 100
all_separate_stocks = {}
for i,d in enumerate(data):
    all_separate_stocks[data.index[i]] = get_stocks_list(d,100,data.index[i])    
pd.DataFrame(all_separate_stocks).to_pickle("plan3.pkl")    

#### plan4

In [ ]:
all_stocks_list = {}
for i,d in enumerate(data):
    print(i)
    shenwan_data = shenwan_instrument_industry(d,data.index[i])
    group_data = shenwan_data.groupby("index_name").apply(lambda x:x.index.tolist())
    all_stocks_list[data.index[i]] = group_data
pd.DataFrame(all_stocks_list).to_pickle("plan4.pkl")    

#### plan5-7

In [23]:
from functools import reduce

In [ ]:
data = pd.read_pickle("plan4.pkl")

In [59]:
def get_extreme_size_stocks(stocks,size,d):
    factor_data = get_factor(stocks,"a_share_market_val",date=d).sort_values().index.tolist()
    stocks_list = [factor_data[:size],factor_data[-size:]]
    return stocks_list
    

In [82]:
size = 1  # 2,5
res = data.apply(lambda x: x.apply(lambda s:get_extreme_size_stocks(s,size,x.name)))
small_size = res.apply(lambda x:x.apply(lambda y:y[0])).apply(lambda x: reduce(lambda z,y:set(z)|set(y),x))
big_size = res.apply(lambda x:x.apply(lambda y:y[1])).apply(lambda x: reduce(lambda z,y:set(z)|set(y),x))
v = pd.concat([small_size,big_size],axis=1).rename(columns={0:"S",1:"B"})
# size 1,2,5 分别对应  plan5,6,7.pkl
v.to_pickle("plan7.pkl")

#### plan8

In [86]:
pd.read_pickle("plan7.pkl").T.apply(lambda x:reduce(lambda z,y:set(z)|set(y),x)).to_pickle("plan8.pkl")